# Créer des variables de décision rapidement.

Dans ce notebook, on va voir comment créer des variables de décision rapidement dans un solveur python (Itérer).

## Sommaire

1. <b>Méthode de base</b>
2. <b>Méthode avec les tableaux</b>
3. <b>Méthode avec le dictionnaire Python Pulp</b>
4. <b>Créer des contraintes plus rapidement avec les dictionnaires pulp.</b>
5. <b>Une astuce pour récupérer les noms des variables de décision.</b>
6. <b>Créer des variables de décision en associant deux tableaux.</b>
    - Méthode d'utilisation
7. <b>Itérer</b>
    - avec le solveur Python Gekko 
    - avec le solveur Python Cplex
    - avec le solveur Python Cvxopt

Etude globale proposée par <b>Estelle Derrien - Github Estelle15000</b>

# 1. Méthode de base

Rappel :
Une variable de décision contient une valeur qui sera ensuite calculée par le solveur pour être la meilleure valeur possible pour minimiser ou maximiser une fonction objectif ( C'est ca l'optimisation)

In [1]:
# La méthode de base :

# Import du solveur
import pulp as p

# On sette le problème
Lp_prob = p.LpProblem('Problem', p.LpMaximize)

# -----------------------------------
# On définit nos variables de décision ci-dessous
# -----------------------------------

x = p.LpVariable("x", lowBound=0, cat='Integer')   # Créer une variable x >= 0
y = p.LpVariable("y", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# On les affiche
print(x)
print(y)

Restricted license - for non-production use only - expires 2024-10-28
No parameters matching '_test' found
x
y


# 2. Méthode avec les tableaux

Dans l'exemple précédent, ca va parce qu'on a que deux variables de décisions, mais qu'en sera-t-il si on en a 100 ?

In [2]:
# La méthode rapide :

import pulp as p

Lp_prob = p.LpProblem('Problem', p.LpMaximize)

# On crée d'abord le nom de nos variables de décision

objets = [
    'objet_1',
    'objet_2',
    'objet_3',
    'objet_4',
    'objet_5',
]

# On crée les variables, celles ci seront binaires. En fait, on itère sur le tableau objets et i est l'index.
x = {i: p.LpVariable(name=f"{i}", lowBound=0, cat='Binary') for i in objets}

# Maintenant, on peut acceder aux variables de cette façon : x['objet_1']

# On peut ensuite créer un deuxième lot de variables de décision, qui cette fois ci, seront entières.

usines = [
    'usine_1',
    'usine_2',
    'usine_3',
    'usine_4',
]

# On crée les variables, celles ci seront entières. En fait, on itère sur le tableau objets et k est l'index.
m = {k: p.LpVariable(name=f"{k}", lowBound=0, cat='Integer') for k in usines}

# On accède ensuite à une variable comme ça, ou avec des boucle for
print(m['usine_1'])


usine_1


# 3. Méthode avec le dictionnaire de Python Pulp


In [3]:
# Import the PuLP lib
from pulp import *

# Créer le type de problème
prob = LpProblem ("MaximiserProfit", LpMaximize)

# La liste de nos produits
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# Ici, on crée nos variable de décision en créant un "dicts".
x = LpVariable.dicts("produits ", produits , 0)

# On visualise notre dicts
print(x)


{'automobile': produits__automobile, 'cycle1': produits__cycle1, 'cycle2': produits__cycle2, 'dragon': produits__dragon, 'nounours': produits__nounours, 'poupee': produits__poupee, 'arc': produits__arc}


# 4. Créer des contraintes plus rapidement avec les dictionnaires

Dans cet exemple ci-dessous, on crée des contraintes à l'aide de dictionnaires python afin d'<b>itérer</b>, pour aller plus vite.

En fait, c'est aussi une association de valeurs à nos variables de décisions, réalisées aux format dictionnaire python, ce qui permet d'écrire plus vite ensuite dans la fonction objectif et dans les contraintes. 

Mais attention, le nombre de variables de décisions doit correspondre au nombre dans les dictionnaires Python , ici, on a 7 variables de décisions, donc dans les dictionnaires, on retrouve cette taille de 7 variables de décision.

Ceci peut faciliter les choses dans le cas ou l'on importe ces données à partir d'une base de données, il devient plus facile de les placer dans un dictionnaire.


In [4]:
# Import the PuLP lib
from pulp import *

# Créer le type de problème, ici, on veut maximiser notre profit de notre usine de jouets
prob = LpProblem ("MaximiserProfit", LpMaximize)

# La liste de nos produits ( des jouets), ca va être les variables de décision, 
# elles pourront prendre une valeur entière, vu que ce sont des objets uniques. 
# Par exemple , produire 15 automobiles, produire 25 arcs etc ...
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# Les bénéfices en EUROS par produits
benefices = {"automobile": 8, "cycle1": 12, "cycle2": 14,"dragon": 3,"nounours":6,"poupee":13,"arc":12}

# Emplois (en kgs)
plastique = {"automobile": 2, "cycle1": 4, "cycle2": 5,"dragon": 3,"nounours":1,"poupee":4,"arc":2}
bois      = {"automobile": 1, "cycle1": 1, "cycle2": 2,"dragon": 2,"nounours":1,"poupee":5,"arc":1}
acier     = {"automobile": 1, "cycle1": 2, "cycle2": 3,"dragon": 3,"nounours":2,"poupee":2,"arc":5}


# Les noms de nos ressources
ressources = {"plastique", "bois", "acier"}

# Les stocks de nos ressources en KG
stocks = {"plastique": 142, "bois ": 117, "acier": 124}

# On crée nos variables, en se basant sur le tableau x ( array en Anglais)
# On stipule que ce sont des variables entières, normal, puisque ce sont des jouets (On ne peut pas avoir 1/2 jouet...).
x = LpVariable.dicts("produits ", produits , lowBound=0, cat='Integer')

# La fonction objectif, Maximiser le bénéfice.
# Ici, on voit qu'on itère à l'aide de notre tableau produits, sur les bénéfices qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([benefices[i] * x[i] for i in produits ]), "MaximiserBenefice" 

# Nos contraintes.
# On respecte notre production sous contrainte de stocks
# Ici, n voit qu'on itère à l'aide de notre tableau produits, sur les stocks qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([plastique[i] * x[i] for i in  produits]) <= 142 ,"MaxPlastique"

# La ligne suivante veut dire : 
# "" Prends chaque valeur du tableau bois indicé par i et multiplie le par la 
# variable de décision x indice i, fait la somme de tout
# et cette somme doit être inférieure à notre stock de bois de 117
prob += lpSum([bois[i]      * x[i] for i in  produits]) <= 117 ,"MaxBois"

prob += lpSum([acier[i]     * x[i] for i in  produits]) <= 124 ,"MaxAcier"

# Production minimale par produits pour les clients : 2 unités
for p in produits:
   prob += x[p] >= 2, f"min production units for product {p}"

# On écrit aussi le probleme dans un fichier
# prob.writeLP ( "JouetsModel.lp")

# On utilise le solver pulp
prob.solve()

# On affiche le statut de la solution
print ("Status:", LpStatus [prob.status])

# Afficher l'optimium de chaques variables produits qui s'exprime en unité construites
for v in prob.variables ():
    print (v.name, "=", v.varValue)


# Le résultat de la fonctioj objectif est ici :
print ("TotalProfit", value (prob.objective))
  

Status: Optimal
produits__arc = 2.0
produits__automobile = 40.0
produits__cycle1 = 2.0
produits__cycle2 = 2.0
produits__dragon = 2.0
produits__nounours = 26.0
produits__poupee = 2.0
TotalProfit 584.0


# 5. Une astuce pour récupérer les noms des variables de décision.

Dans cet exemple, ci-dessous, on voit que l'auteur écrit directement les caractéristiques des variables de décision dans des dictionnaires python, puis il récupère les noms des variables de décision à l'aide de la commande
items = list(sorted(v.keys())).

Du coup, ca va encore plus vite pour créer les variables ensuite dans la commande :
x = LpVariable.dicts('x', items, lowBound=0, upBound=1, cat=LpInteger)

In [5]:
# knapsack-pulp.py

from pulp import *

v = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
limit = 14
items = list(sorted(v.keys()))

# Create model
m = LpProblem("Knapsack", LpMaximize)

# Variables
x = LpVariable.dicts('x', items, lowBound=0, upBound=1, cat=LpInteger)

# 6. Créer des variables de décision en associant deux tableaux.


Dans cet exemple, on voit que l'auteur de Python Pulp crée les variables de décision en associant 2 tableaux : les fournisseurs et les clients.

Ca permet de créer toutes les solutions possibles à l'aide de la boucle FOR.

On s'en sert, après , pendant l'optimisation

In [6]:
import pulp as p

# Variables de décision
fournisseurs = ['A','B']
clients = ['1','2','3','4','5']
costs = [  
# clients   # 1  2  3  4  5
                [3, 1, 3, 2,9],      # A   fournisseurs
                [25, 15, 32, 22,8],  # B
]

# C'est un problème de minimisation
Problem = p.LpProblem('optimisation_transport',p.LpMinimize)

# Créer une liste de toutes les routes possibles
Routes = [(w, b) for w in fournisseurs  for b in clients]

# Créer un dictionnaire de variables de décisions des routes
vars = p.LpVariable.dicts("Route", (fournisseurs , clients), 0, None, p.LpBinary)

# Créer un dictionnaire des coûts de chaque routes
cost = p.makeDict([fournisseurs,clients], costs, 0)

# Fonction objectif
Problem += p.lpSum([cost[w][b]  * vars[w][b]  for (w, b) in Routes])

# Contrainte
Problem += (p.lpSum([ vars[w][b]  for (w, b) in Routes]) >= 5 , "minimum_de_5_routes")

# On résouds
Problem.solve()
 
# On imprime le résultat
print('Statut:', p.LpStatus[Problem.status])
print('Cout total minimisé = ', p.value(Problem.objective))
 
for i in Problem.variables():
    if i.varValue > 0:
        print('Choix de la route :',i.name, '=', i.varValue)


Statut: Optimal
Cout total minimisé =  17.0
Choix de la route : Route_A_1 = 1.0
Choix de la route : Route_A_2 = 1.0
Choix de la route : Route_A_3 = 1.0
Choix de la route : Route_A_4 = 1.0
Choix de la route : Route_B_5 = 1.0


La syntaxe des problèmes linéaire peut être organisée différemment selon les auteurs des programmes, voici l'exemple de la production agrégée de Aaon Stubberfield ou il associe aussi des tableaux pour créer des variables de décision, mais de façon différente que le premier exemple .

D'autant plus qu'un code est toujours l'application d'un modèle mathématique écrit, mais là, cela devient difficile, il faut s'habituer et prendre le temps de lire et de comprendre. On est censé savoir faire le va et vient entre le modèle mathématique et le code, et vice-versa.


In [7]:
from pulp import *
demand = {'A':[5,0,0],'B':[8,7,6]}
costs = {'A':[20,17,18],'B':[15,16,15]}

# On crée le problème
model = LpProblem("Aggregate_Production_Planning",LpMinimize)

# On définit les variables
time = [0, 1, 2]
prod = ['A','B']

# On crée un dictionnaire de toutes les variables de décision de productions possibles et leur temps
#### C'est là que l'on crée des variables de décision en associant les 2 tableaux.***
X = LpVariable.dicts("prod", [(p, t) for p in prod for t in time],lowBound=0, cat="Integer")
# On regarde l'association créee :
print(X)

# On crée la fonction objectif : minimiser les coûts
model += lpSum([costs[p][t] * X[(p, t)] for p in prod for t in time])

# On définit la contrainte que la production >= demande
for p in prod:
    for t in time:
        model += X[(p, t)] >= demand[p][t] 

# Résoudre
model.solve()

# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Coût total = ", value(model.objective))

{('A', 0): prod_('A',_0), ('A', 1): prod_('A',_1), ('A', 2): prod_('A',_2), ('B', 0): prod_('B',_0), ('B', 1): prod_('B',_1), ('B', 2): prod_('B',_2)}
prod_('A',_0) = 5.0
prod_('A',_1) = 0.0
prod_('A',_2) = 0.0
prod_('B',_0) = 8.0
prod_('B',_1) = 7.0
prod_('B',_2) = 6.0
Coût total =  422.0


# 7. Itérer avec Python Gekko , le  solveur non  linéaire.

C'est différent d'avec Pulp .

Lien : https://stackoverflow.com/questions/64542594/how-could-constraints-be-dynamically-constructed-in-gekko

Voici un exemple de réduction de coût avec une itération dans la contrainte de temps de travail :

In [8]:
from gekko import GEKKO

# stored as list
my_vars = ['x1','x2']
# stored as dictionaries
Cost = {'x1':100,'x2':125}
Min = {'x1':0,'x2':0}
Max = {'x1':70,'x2':40}
Work = {'x1':50,'x2':50}

LP = GEKKO(remote=False)


va = LP.Array(LP.Var, (len(my_vars)))  # array

# Le truc qui diffère , en fait il crée un dictionnaire à l'aide du tableau créé juste avant :
vd = {}                                # dictionary
for i,xi in enumerate(my_vars):
    vd[xi] = va[i]
    vd[xi].lower = Min[xi]
    vd[xi].upper = Max[xi]


# Fonction coût
LP.Minimize(LP.sum([Cost[xi]*vd[xi] for xi in my_vars])) 


# On voit que l'itération fonctionne !
LP.Equation(LP.sum([Work[xi]*vd[xi] for xi in my_vars])>=200)


LP.solve(disp=False)

# On affiche le résultat
for xi in my_vars:
    print(xi,vd[xi].value[0])
print ('Cost: ' + str(LP.options.OBJFCNVAL))

x1 3.9999999998
x2 1.8225924919e-09
Cost: 400.00000021


# 7b. Itérer avec le solveur Python Cplex